In [38]:
import warnings
warnings.filterwarnings("ignore")

import sys  
!{sys.executable} -m pip install contractions==0.0.18
!pip install pyspellchecker
!pip install contractions
!pip install tensorflow==2.7.0 --user
quiet = True

In [16]:
import tensorflow
print(tensorflow.__version__)

2.7.0


In [17]:
import os
import random
import pickle
import numpy as np

import tkinter as tk
from tkinter import *
from tkinter import filedialog

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

text_contents=dict()

In [18]:
from NLP_text_preprocess import PreProcessing
import config

In [29]:
os.chdir(r"C:\Users\hi\Desktop\Capstone Project\Jan-G4---NLP-Chatbot")
from keras.models import load_model
model = load_model("transformer_model")

In [30]:
with open(r"parameters.pickle", "rb") as f:
    lab_en, tokenizer_data = pickle.load(f)

In [31]:
#os.chdir(r"F:\Great learning\Capstone")
import pandas as pd
industry_df = pd.read_csv("industry_df_with_stopwords.csv")
usr_txt = industry_df['Description_preprocessed'][0]

In [32]:
usr_txt

'while removing the drill rod of the jumbo 08 for maintenance the supervisor proceeds to loosen the support of the intermediate centralized to facilitate the removal seeing this the mechanic support one end on the drill of the equipment to pull with both hand the bar and accelerate the removal from this at this moment the bar slide from it point of support and tightens the finger of the mechanic between the drilling bar and the beam of the jumbo'

In [34]:
import warnings
warnings.filterwarnings("ignore")
pp = PreProcessing(to_lower = config.to_lower,remove_url=config.remove_url,
                   remove_time = config.remove_time,expand_contraction =config.expand_contraction,
                   remove_special_character=config.remove_special_character,
                   remove_punctuation=config.remove_punctuation,
                   remove_whitespace=config.remove_whitespace,
                   keep_alpha_numeric = False,
                   check_spelling=config.check_spelling,
                   remove_stopword=False,
                   lemmatize_word=config.lemmatize_word)

pre_txt = pp.preprocess(usr_txt)
tok_txt = tokenizer_data.texts_to_sequences([pre_txt])
pad_txt = pad_sequences(tok_txt, padding='post', maxlen=200)
prediction = model.predict(pad_txt, verbose=0)
pred_index = np.argmax(prediction,axis=1)
result = lab_en.inverse_transform(pred_index)[0]
print('The Potential accident level for this accident is {}'.format(result))

The Potential accident level for this accident is IV


Creating functions for all the operations we tend to do n chatbot.

In [35]:
def greet_res(text):
    text=text.lower()
    bot_greet=['hi','hello','hola','hey','howdy']
    usr_greet=['hi','hey','hello','hola','greetings','wassup','whats up']
    for word in text.split():
        if word in usr_greet:
            return random.choice(bot_greet)
# bot response
def bot_ress(usr_input):
    warnings.filterwarnings("ignore")
    with open(r"parameters.pickle", "rb") as f:
          lab_en, tokenizer_data = pickle.load(f)
    pp = PreProcessing(to_lower = config.to_lower,remove_url=config.remove_url,
                       remove_time = config.remove_time,expand_contraction =config.expand_contraction,
                       remove_special_character=config.remove_special_character,
                       remove_punctuation=config.remove_punctuation,
                       remove_whitespace=config.remove_whitespace,
                       keep_alpha_numeric = False,
                       check_spelling=config.check_spelling,
                       remove_stopword=False,
                       lemmatize_word=config.lemmatize_word)

    pre_txt = pp.preprocess(usr_input)
    tok_txt = tokenizer_data.texts_to_sequences([pre_txt])
    pad_txt = pad_sequences(tok_txt, padding='post', maxlen=200)
    prediction = model.predict(pad_txt, verbose=0)
    pred_index = np.argmax(prediction,axis=1)
    result = lab_en.inverse_transform(pred_index)[0]
    bot_res = 'The Potential accident level for this accident is {}'.format(result)
    return bot_res

def widget_get():
    text_widget = root.nametowidget(textcon)
    return text_widget.get('1.0','end-1c')

In [12]:
#os.chdir(r"F:\Great learning\Capstone")

Creating Function for all the events need to do in our chatbot.

In [36]:
def saveas(event=None):
    global file_path,filename
    file_path= filedialog.asksaveasfilename( defaultextension=".txt")
    try:
        filename=os.path.basename(file_path)
        root.title(f"Chat Bot - {filename}")
        content=widget_get()
        with open(file_path ,"w") as file:
            file.write(content)
            text_contents[str(textcon)]=hash(content)
            print("Operation successfull")
    except(FileNotFoundError):
        print("Operation not successfull")
        return None
file_path =None

def save(event=None):
    global file_path,filename
    try:
        if(file_path is None):
            file_path = filedialog.asksaveasfilename(defaultextension=".txt")
        filename=os.path.basename(file_path)
        root.title(f"Chat Bot - {filename}")
        content=widget_get()
        with open(file_path ,"w") as file:
            file.write(content)
            text_contents[str(textcon)] = hash(content)
            print("Operation successfull")
    except(FileNotFoundError):
        print("Operation not successfull")
        return None

def new(event=None):
    textcon.delete('2.0', 'end-1c')
    global  file_path,filename
    file_path = None
    content = widget_get()
    text_contents[str(textcon)] = hash(content)
    filename=None

def clear(event=None):
    textcon.delete('2.0', 'end-1c')
    content = widget_get()
    text_contents[str(textcon)] = hash(content)
    
def fopen(event=None):
    global file_path,filename
    file_path = filedialog.askopenfilename(defaultextension=".txt")
    try:
        filename = os.path.basename(file_path)
        root.title(f"Chat Bot - {filename}")
        text_widget = root.nametowidget(textcon)
        with open(file_path, "r") as file:
            content=file.read()
            textcon.delete('1.0', 'end-1c')
            text_contents[str(textcon)] = hash(content)
            text_widget.insert(END,content)
            print("Operation successfull")
    except(FileNotFoundError):
        print("Operation not successfull")
        return None

exit_list = ['exit','break','quit','see you later','chat with you later','end the chat','bye','ok bye']

def send(event=None):
    usr_input = message.get()
    usr_input = usr_input.lower()
    textcon.insert(END, f'User: {usr_input}'+'\n','usr')
    if usr_input in exit_list:
        textcon.config(fg='Yellow')
        textcon.insert(END,"Bot:Ok bye! Chat with you later\n")
        return root.destroy()
    else:
        textcon.config(fg='yellow')
        if greet_res(usr_input) != None:
            lab=f"Bot: {greet_res(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)
        else:
            lab = f"Bot: {bot_ress(usr_input)}"+'\n'
            textcon.insert(END,lab)
            mes_win.delete(0,END)

### Creating Chatbot:

In [37]:
root=tk.Tk()
filename="Untitled.txt"
root.title(f"Chat Bot - Untitled.txt")
root.geometry('500x400')

root.resizable(False, False)
main_menu=Menu(root)
file_menu=Menu(root)
file_menu.add_command(label='Open  <Ctrl+O>',command=fopen)
file_menu.add_command(label='New  <Ctrl+N>',command=new)
file_menu.add_command(label='Save  <Ctrl+S>',command=save)
file_menu.add_command(label='Save as <Ctrl+Shift+S>',command=saveas)
edit_menu=Menu(root)
edit_menu.add_command(label='Clear  <Delete>',command=clear)
main_menu.add_cascade(label="File",menu=file_menu)
main_menu.add_cascade(label="Edit",menu=edit_menu)
main_menu.add_command(label="Quit",command=root.destroy)
root.config(menu=main_menu)

message=tk.StringVar()
chat_win=Frame(root,bd=1,bg='black',width=50,height=8)
chat_win.place(x=6,y=6,height=300,width=480)

textcon=tk.Text(chat_win,bd=1,bg='black',width=50,height=8)
textcon.pack(fill="both",expand=True)

mes_win=Entry(root,width=30,xscrollcommand=True,textvariable=message)
mes_win.place(x=6,y=310,height=60,width=366)
mes_win.focus()

textcon.config(fg='yellow')
textcon.tag_config('usr',foreground='white')
textcon.insert(END,"Bot: This is Jarvis to assist you in Industrial safety!\n\n")
mssg=mes_win.get()

button=Button(root,text='Send',bg='grey',activebackground='orange',command=send,width=12,height=5,font=('Arial'))
button.place(x=376,y=310,height=60,width=110)

scrollbar=tk.Scrollbar(textcon)
scrollbar.pack(fill='y')
scrollbar.place(relheight = 1,relx = 1)
scrollbar.config(command = textcon.yview)

content = widget_get()
text_contents[str(textcon)] = hash(content)
root.bind('<Control-s>',save,file_menu)
root.bind('<Control-Shift-s>',saveas,file_menu)
root.bind('<Return>', send,button)
root.bind('<Control-n>', new,file_menu)
root.bind('<Delete>', clear,edit_menu)
root.bind('<Control-o>', fopen,file_menu)
root.mainloop()